<a href="https://colab.research.google.com/github/andremonroy/stanWeinstein/blob/main/RS_Sectores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Primero calculamos el RS rating por sector y lo exporto a Google Sheets "RS dashboard"

In [ ]:
# CELDA 1 – Instalación
!pip install --quiet yfinance gspread oauth2client

In [ ]:
# CELDA 2 – Configuración Google Sheets
import pandas as pd, numpy as np, yfinance as yf, datetime, gspread
# from oauth2client.service_account import ServiceAccountCredentials
from google.colab import auth

# -> Coloca tu ID
spreadsheet_id = "19WW_XIkvM0VU1W_NYDBEZwU6NxYvkTAb2Qn_E7ONAyw"
sheet_name     = "RS sectores"

# Authenticate using google.colab.auth
auth.authenticate_user()

# Use the authenticated user's credentials
import google.auth
import google.auth.transport.requests
creds, project = google.auth.default()

# Authorize gspread with the obtained credentials
client = gspread.authorize(creds)

sheet  = client.open_by_key(spreadsheet_id).worksheet(sheet_name)

In [ ]:
# CELDA 3 – Parámetros y descarga de precios semanales

# ETFs sectoriales megatendencia y  S&P 500
sector_etfs = [
    'XLK','XLV','XLF','XLY','XLP','XLI','XLB','XLE','XLU','XLRE','XLC'
]

# ETFs megatendencias
megatrend_etfs = [
    # 1. AI & Big Data
    'AIQ','BOTZ','ROBO','IRBO','ARKQ',
    # 2. Ciberseguridad y Defensa Digital
    'BUG','IHAK','CIBR',
    # 3. Infraestructura Digital y Data Centers
    'DTCR','PAVE',
    # 4. Energías limpias y Transición Energética
    'RAYS','WNDY','CTEC','RNRG','AQWA','HYDR',
    # 5. Genómica, Salud Digital y Biotecnología
    'GNOM','HEAL','XBI',
    # 6. Demografía y Hábitos de Consumo
    'MILN','AGNG',
    # 7. Reshoring Industrial y Geopolítica
    'AMER','PAVE',  # PAVE ya está en Infraestructura, pero se repite en temática
    # 8. Blockchain, Cripto, FinTech
    'BLOK','FINX','BWEB',
    # 9. Smart Cities, IoT y Real Estate Tech
    'SNSR','PTEC',
    # 10. Espacio, Tecnología Aeroespacial
    'ARKX','ITA',
    # 11. E-sports y Entretenimiento Digital
    'HERO','SOCL'
]

# Unimos ambas listas (eliminando duplicados)
etfs = list(dict.fromkeys(sector_etfs + megatrend_etfs))

benchmark = 'SPY'
today     = datetime.date.today()
start     = today - datetime.timedelta(weeks=60)

data = yf.download(etfs + [benchmark], start=start, end=today, interval='1wk')
data = data.dropna(how='all')  # quita filas vacías

# Eliminar ETFs que no tengan datos de precio (todas NaN) para evitar columnas totalmente vacías
available_etfs = [t for t in etfs if t in data['Close'].columns and data['Close'][t].notna().any()]

missing = sorted(list(set(etfs) - set(available_etfs)))
if missing:
    print("Se eliminaron (sin datos):", missing)

# Reemplazamos la lista 'etfs' por las que sí tienen datos
etfs = available_etfs
# --------------------------------



/tmp/ipython-input-4003868101.py:41: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(etfs + [benchmark], start=start, end=today, interval='1wk')
[*********************100%***********************]  43 of 43 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AMER']: YFPricesMissingError('possibly delisted; no price data found  (1wk 2024-06-18 -> 2025-08-12)')


Se eliminaron (sin datos): ['AMER']


In [ ]:
# ----- CELDA 4 – Cálculo de retornos relativos y RS escalado 0-100 (por grupo) -----

# Ventanas y pesos
windows  = {'3m': 13, '6m': 26, '9m': 39, '12m': 52}
weights  = {'3m': 0.40, '6m': 0.30, '9m': 0.20, '12m': 0.10}

# Filtrar solo ETFs que realmente tienen datos
sector_available = [t for t in sector_etfs if t in etfs]
mega_available   = [t for t in megatrend_etfs if t in etfs]

def compute_scores(etf_list):
    """Calcula el score ponderado (sin escalar) para la lista etf_list."""
    if not etf_list:
        return pd.Series(dtype=float)
    scores = pd.Series(0.0, index=etf_list)
    for lbl, wks in windows.items():
        rel = data['Close'].pct_change(wks)
        # ETF / benchmark
        rel_rs = rel[etf_list].div(rel[benchmark], axis=0)
        latest = rel_rs.iloc[-1]
        scores += latest * weights[lbl]
    return scores

def scale_scores(scores):
    """Escalado 0-100 seguro (si todos iguales devuelve 50 neutral)."""
    if scores.empty:
        return pd.Series(dtype=float)
    mn, mx = scores.min(), scores.max()
    if mx - mn == 0:
        return pd.Series(50.0, index=scores.index)
    return (100 * (scores - mn) / (mx - mn)).round(1)

# Función de ajuste lineal para igualar referencia externa
def ajustar_rs(rs_manual):
    return (0.85 * rs_manual + 6.1).round(1)

# -------------------
# Cálculo para Core
# -------------------
scores_core      = compute_scores(sector_available)
rs_scaled_core   = scale_scores(scores_core)
rs_ajustado_core = ajustar_rs(rs_scaled_core)

# Alinear índices Core
rs_scaled_core, rs_ajustado_core = rs_scaled_core.align(rs_ajustado_core, join='inner')

# -------------------
# Cálculo para Megatendencias
# -------------------
scores_mega      = compute_scores(mega_available)
rs_scaled_mega   = scale_scores(scores_mega)
rs_ajustado_mega = ajustar_rs(rs_scaled_mega)

# Alinear índices Mega
rs_scaled_mega, rs_ajustado_mega = rs_scaled_mega.align(rs_ajustado_mega, join='inner')

# Diagnóstico rápido
print("ETFs Core procesados:", rs_scaled_core.index.tolist())
print("ETFs Megatendencias procesados:", rs_scaled_mega.index.tolist())


ETFs Core procesados: ['XLK', 'XLV', 'XLF', 'XLY', 'XLP', 'XLI', 'XLB', 'XLE', 'XLU', 'XLRE', 'XLC']
ETFs Megatendencias procesados: ['AIQ', 'BOTZ', 'ROBO', 'IRBO', 'ARKQ', 'BUG', 'IHAK', 'CIBR', 'DTCR', 'PAVE', 'RAYS', 'WNDY', 'CTEC', 'RNRG', 'AQWA', 'HYDR', 'GNOM', 'HEAL', 'XBI', 'MILN', 'AGNG', 'PAVE', 'BLOK', 'FINX', 'BWEB', 'SNSR', 'PTEC', 'ARKX', 'ITA', 'HERO', 'SOCL']


/tmp/ipython-input-741833016.py:17: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rel = data['Close'].pct_change(wks)
/tmp/ipython-input-741833016.py:17: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rel = data['Close'].pct_change(wks)
/tmp/ipython-input-741833016.py:17: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rel = data['Close'].pct_change(wks)
/tmp/ipython-input-741833016.py:17: Futu

In [ ]:
# ----- CELDA 5 – Semáforo y subida a Google Sheets (Core + Megatendencias por separado) -----

def light(v):
    try:
        if v is None or (isinstance(v, float) and np.isnan(v)):
            return 'blanco'
        v = float(v)
    except:
        return 'blanco'
    if v >= 80:
        return 'verde fuerte'
    elif v >= 70:
        return 'verde claro'
    elif v >= 40:
        return 'amarillo'
    else:
        return 'rojo'

# Diccionario de nombres (Core + Megatendencias)
etf_nombres = {
    'XLF': 'Financials','XLK': 'Technology','XLY': 'Consumer Discretionary','XLP': 'Consumer Staples',
    'XLV': 'Health Care','XLI': 'Industrials','XLE': 'Energy','XLU': 'Utilities',
    'XLB': 'Materials','XLRE': 'Real Estate','XLC': 'Communication Services',
    'AIQ':'AI & Big Data','BOTZ':'Robótica e IA','ROBO':'Robótica Global','IRBO':'IA Multisector',
    'ARKQ':'Tecnología Autónoma y Robótica','BUG':'Ciberseguridad','IHAK':'Ciberseguridad ESG',
    'CIBR':'Defensa Tecnológica','DTCR':'Infraestructura Digital','PAVE':'Infraestructura y Energía',
    'RAYS':'Energía Solar','WNDY':'Energía Eólica','CTEC':'Tecnología Limpia',
    'RNRG':'Productores Energía Renovable','AQWA':'Agua Limpia','HYDR':'Hidrógeno',
    'GNOM':'Genómica & Biotech','HEAL':'Salud e Innovación','XBI':'Biotecnología General',
    'MILN':'Consumo Millennial','AGNG':'Envejecimiento Población','AMER':'Manufacturing USA',
    'BLOK':'Blockchain','FINX':'FinTech General','BWEB':'Web3 y Descentralización',
    'SNSR':'Internet of Things','PTEC':'PropTech / Real Estate Digital',
    'ARKX':'Espacio y Defensa','ITA':'Aeroespacial','HERO':'Videojuegos y E-sports',
    'SOCL':'Media y Entretenimiento Digital'
}

# Construir DataFrame desde las Series escaladas de cada grupo
def build_df_from_series(rs_scaled_series, rs_ajustado_series):
    if rs_scaled_series.empty:
        return pd.DataFrame(columns=['ETF','Sector','RS','RS Ajustado','Semáforo'])
    available = list(rs_scaled_series.index)
    df = pd.DataFrame({
        'ETF': available,
        'Sector': [etf_nombres.get(t, t) for t in available],
        'RS': rs_scaled_series.loc[available].values,
        'RS Ajustado': rs_ajustado_series.loc[available].values
    })
    df['Semáforo'] = df['RS Ajustado'].apply(light)
    df = df[['ETF','Sector','RS','RS Ajustado','Semáforo']].sort_values('RS Ajustado', ascending=False)
    return df

df_core = build_df_from_series(rs_scaled_core, rs_ajustado_core)
df_mega = build_df_from_series(rs_scaled_mega, rs_ajustado_mega)

# Diagnóstico rápido: qué tickers originales quedaron fuera por falta de datos
missing_core = [t for t in sector_etfs if t not in df_core['ETF'].tolist()]
missing_mega = [t for t in megatrend_etfs if t not in df_mega['ETF'].tolist()]
if missing_core:
    print("Core (omitidos por no tener datos/calculo):", missing_core)
if missing_mega:
    print("Megatendencias (omitidos por no tener datos/calculo):", missing_mega)

# Export CSVs (opcional)
df_core[['Sector','RS Ajustado']].to_csv("sectores_rs_core.csv", index=False)
df_mega[['Sector','RS Ajustado']].to_csv("sectores_rs_mega.csv", index=False)
# También un CSV combinado (si lo usas en otro notebook)
pd.concat([df_core, df_mega]).to_csv("sectores_rs.csv", index=False)

# Preparar filas seguras para Google Sheets (no NaN ni inf)
def df_to_rows_safe(df):
    df2 = df.copy()
    df2 = df2.replace([np.inf, -np.inf], np.nan).fillna('')
    return [list(df2.columns)] + df2.values.tolist()

rows = [["ETFs Principales"]] + df_to_rows_safe(df_core) \
     + [[""]] \
     + [["ETFs Megatendencias"]] + df_to_rows_safe(df_mega)

# Subir a Google Sheets: soporte para InteractiveSheet (si lo hubieras sobrescrito)
try:
    if hasattr(sheet, 'clear') and hasattr(sheet, 'update'):
        sheet.clear()
        sheet.update(rows)
    else:
        ws = client.open_by_key(spreadsheet_id).worksheet(sheet_name)
        ws.clear()
        ws.update(rows)
    print("¡Dashboard actualizado con Core y Megatendencias!")
except Exception as e:
    print("Error subiendo a Google Sheets:", e)
    # fallback: imprimir preview
    print("Preview de las primeras filas:")
    for r in rows[:10]:
        print(r)

# Mostrar en notebook para verificación rápida
print("Top Core (vista rápida):")
display(df_core.head(12))
print("Top Megatendencias (vista rápida):")
display(df_mega.head(12))


ValueError: All arrays must be of the same length

In [ ]:
'''from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)
'''